#P0. Introducción -PLN

---

En la creación de redes neuronales necesitamos dos tipos de IA, para reconocer patrones o generar nuevos:

*   Las que no tienen memoria, identifica el patrón y ya!...ejemplo las de visión artificial
*   Las de memoria corta (Long Short Term Memory)...PLN
*   Las que requieren mucha memoria (aprenden casi todo...BERT)...PLN y visión artificial.



**Caso de estudio: generación de texto**

---


Cualquier dato que se necesite procesar (sonido, imágenes, texto) primero debe ser convertido en un tensor numérico, un paso llamado “vectorización” (One-hot Encoding y WordEmbedding) de datos (y en nuestro ejemplo previamente las letras deben ser pasadas a valores numéricos 

Para este ejemplo usaremos “*Character level language model*” propuesto por Andrej Karpathy en su artículo "*The Unreasonable Effectiveness of Recurrent Neural Networks*"(y parcialmente basado en su implementado en el tutorial "*Generate text with an RNN*" de la web de TensorFlow:

Consiste en darle a la RNN una palabra y se le pide que modele la distribución de probabilidad del siguiente carácter que le correspondería a la secuencia de caracteres anteriores:

Como ejemplo, supongamos que solo tenemos un vocabulario de cuatro letras posibles [“a”,”h”,”l”,”o”], y queremos entrenar a una RNN en la secuencia de entrenamiento “hola”. Esta secuencia de entrenamiento es, de hecho, una fuente de 3 ejemplos de entrenamiento por separado: La probabilidad de “o” debería ser verosímil dada el contexto de “h”, “l” debería ser verosímil en el contexto de “ho”, y finalmente “a” debería ser también verosímil dado el contexto de “hol”.


---
*   https://unipython.com/generacion-de-textos-con-inteligencia-artificial/
*    https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa (https://www.youtube.com/watch?v=kaQCdv46OBA&ab_channel=JeffHeaton)

#Ejemplo: generar texto de cuentos, usando Keras

#Instalando librerias CUDA

In [1]:
!nvcc --version  #Version de CUDA en la maquina virtual

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


##P0. importar librerias

In [2]:
import tensorflow as tf
import timeit               #para medir tiempos
import numpy as np
import pandas as pd 
import os
import time
import sys

uso de GPU para entrenar en tensorflow

---
https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa


#Uso de GPU para entrenar en tensorflow

In [3]:
print("Tensorflow Version: ", tf.__version__)
print("Dispositivos disponibles para entrenar: ", tf.config.list_physical_devices())
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Encontrada la GPU: {}'.format(device_name))

Tensorflow Version:  2.7.0
Dispositivos disponibles para entrenar:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Encontrada la GPU: /device:GPU:0


#Dejando activo la GPU

In [4]:
#tf.device('/gpu:0') #activando la CPU
tf.device('/device:GPU:0') #activando la GPU

##P0. Descarga y preprocesado de los datos

In [5]:
fileDL= tf.keras.utils.get_file('Origen.txt','https://raw.githubusercontent.com/Julianjimenez98/DeepLearning/main/Origen.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')
texto = texto.lower()
#texto = texto.strip() utf-8-sig

65536/55067 [===================================] - 0s 0us/step


#P0.1 Limpiar/normalizar el texto

In [6]:
import re
from unicodedata import normalize
#pasa todo a minuscula
texto     = texto.lower()
#reemplazar tildes por letras similares sin tildes
transfor  = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
texto     = normalize('NFKC', normalize('NFKD', texto).translate(transfor))
#quitar saltos de linea
texto      = texto.strip()
texto      = re.sub('\r|\n', ' ',texto)
#quitar espacios dobles
texto      = re.sub(' +', ' ', texto)
#quitando caracteres especiales
texto = re.sub(r"[^a-zA-Z0-9]+"," ",texto)
print(texto)

 el profesor robert langdon levanto la mirada hacia el perro de doce metros de altura que habia en medio de la plaza el pelaje del animal estaba formado por un tapiz organico de hierba y flores aromaticas estoy haciendo un esfuerzo por apreciarte penso de veras langdon permanecio un rato examinando la criatura y luego siguio adelante por una pasarela suspendida esta desembocaba en una extensa escalinata cuyos desiguales escalones parecian querer impedir que el visitante recien llegado pudiera mantener su ritmo y su zancada habituales mision cumplida decidio despues de casi estar a punto de tropezar en dos ocasiones al llegar al pie de la escalera se detuvo de golpe y se quedo mirando la enorme estatua que tenia delante ahora si que lo he visto todo ante sus ojos se elevaba una enorme viuda negra sus esbeltas patas de hierro sostenian un cuerpo bulboso que se encontraba a unos diez metros de altura del abdomen del animal colgaba un saco de huevos hecho con una malla metalica y en cuyo i

##P1. entendiendo el texto

In [7]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:50724 caracteres
el texto esta compuesto de estos :34 caracteres
[' ', '1', '3', '4', '5', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


##P2. pasar el texto a números

---
as redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos “tablas de traducción”: una de caracteres a números y otra de números a caracteres

In [8]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: ' el profesor robert langdon levanto la mirada hacia el perro de doce metros de altura que habia en m'
array([ 0, 12, 19,  0, 23, 25, 22, 13, 12, 26, 22, 25,  0, 25, 22,  9, 12,
       25, 27,  0, 19,  8, 21, 14, 11, 22, 21,  0, 19, 12, 29,  8, 21, 27,
       22,  0, 19,  8,  0, 20, 16, 25,  8, 11,  8,  0, 15,  8, 10, 16,  8,
        0, 12, 19,  0, 23, 12, 25, 25, 22,  0, 11, 12,  0, 11, 22, 10, 12,
        0, 20, 12, 27, 25, 22, 26,  0, 11, 12,  0,  8, 19, 27, 28, 25,  8,
        0, 24, 28, 12,  0, 15,  8,  9, 16,  8,  0, 12, 21,  0, 20])


#P2.1 Exportar vocablos y matriz de numerica

In [9]:
rows=[]
columns=['num','vocab']
for i, voc in enumerate(vocab):
  #print(i,'-->', voc)
  rows.append([i,voc])
df= pd.DataFrame(columns=['num','vocab'],data=rows)
df.head(10)
df.to_csv('data_vocab.csv',index=False)

##P3. preparar los datos para ser usados en la RNN

In [10]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=50
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

' el profesor robert langdon levanto la mirada hacia'
' el perro de doce metros de altura que habia en med'
'io de la plaza el pelaje del animal estaba formado '
'por un tapiz organico de hierba y flores aromaticas'
' estoy haciendo un esfuerzo por apreciarte penso de'
' veras langdon permanecio un rato examinando la cri'
'atura y luego siguio adelante por una pasarela susp'
'endida esta desembocaba en una extensa escalinata c'
'uyos desiguales escalones parecian querer impedir q'
'ue el visitante recien llegado pudiera mantener su '


###P3.1 separar los datos en agrupamientos (batches)

In [11]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  ' el profesor robert langdon levanto la mirada haci'
Target data:  'el profesor robert langdon levanto la mirada hacia'


In [12]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((50,), (50,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 50), (64, 50)), types: (tf.int64, tf.int64)>


##P4.Construcción del modelo RNN

---
Para construir el modelo usaremos tf.keras.Sequential. Usaremos una versión mínima de RNN, que contenga solo una capa LSTM y 3 capas.


In [13]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss=lossy,
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           8704      
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 34)            34850     
                                                                 
Total params: 5,290,530
Trainable params: 5,290,530
Non-trainable params: 0
_________________________________________________________________


###P4.1 Creando chekpoints

---
una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.

---
los crearemos en google drive para mejorar la capacidad de reentrenamiento de la red


In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
checkpoint_dir='/content/gdrive/MyDrive/Colab Notebooks//checkpoints_test/'
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")

cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')

###P4.2 entrenando

####P4.2a entrenando para usar chekpoints

In [16]:
EPOCHS=500 
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])

Epoch 1/500
15/15 [==============================] - ETA: 0s - loss: 3.0520 - accuracy: 0.1496
Epoch 00001: loss improved from inf to 3.05196, saving model to /content/gdrive/MyDrive/Colab Notebooks//checkpoints_test/cp_0001.ckpt
15/15 [==============================] - 8s 229ms/step - loss: 3.0520 - accuracy: 0.1496
Epoch 2/500
15/15 [==============================] - ETA: 0s - loss: 2.7228 - accuracy: 0.1887
Epoch 00002: loss improved from 3.05196 to 2.72275, saving model to /content/gdrive/MyDrive/Colab Notebooks//checkpoints_test/cp_0002.ckpt
15/15 [==============================] - 2s 143ms/step - loss: 2.7228 - accuracy: 0.1887
Epoch 3/500
15/15 [==============================] - ETA: 0s - loss: 2.5909 - accuracy: 0.2649
Epoch 00003: loss improved from 2.72275 to 2.59087, saving model to /content/gdrive/MyDrive/Colab Notebooks//checkpoints_test/cp_0003.ckpt
15/15 [==============================] - 2s 147ms/step - loss: 2.5909 - accuracy: 0.2649
Epoch 4/500
15/15 [================


#####4.2a-1 entrenando desde un checkpoint

---
Desde la carpeta que optamos guardar los checkpoints

*   el archivo .data es el archivo que contiene nuestras variables de entrenamiento y vamos a ir tras él.
*   el archivo checkpoint, simplemente mantiene un registro de los últimos archivos de punto de control guardados




In [17]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model2=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

/content/gdrive/MyDrive/Colab Notebooks//checkpoints_test/cp_0461.ckpt


In [18]:
# cargamos los pesos al nuevo modelo (estos valores tienes una variación de un 10%)
model2.load_weights(latest)
# continuamos el entrenamiento desde el checkpoint en que quedamos
history_V2=model2.fit(dataset, 
                    epochs=200, 
                    verbose=1,
                    callbacks=[cp_callback])

Epoch 1/200
15/15 [==============================] - ETA: 0s - loss: 0.1303 - accuracy: 0.9602
Epoch 00001: loss did not improve from 0.12800
15/15 [==============================] - 3s 95ms/step - loss: 0.1303 - accuracy: 0.9602
Epoch 2/200
15/15 [==============================] - ETA: 0s - loss: 0.1293 - accuracy: 0.9605
Epoch 00002: loss did not improve from 0.12800
15/15 [==============================] - 1s 90ms/step - loss: 0.1293 - accuracy: 0.9605
Epoch 3/200
15/15 [==============================] - ETA: 0s - loss: 0.1286 - accuracy: 0.9607
Epoch 00003: loss did not improve from 0.12800
15/15 [==============================] - 1s 89ms/step - loss: 0.1286 - accuracy: 0.9607
Epoch 4/200
15/15 [==============================] - ETA: 0s - loss: 0.1318 - accuracy: 0.9599
Epoch 00004: loss did not improve from 0.12800
15/15 [==============================] - 1s 89ms/step - loss: 0.1318 - accuracy: 0.9599
Epoch 5/200
15/15 [==============================] - ETA: 0s - loss: 0.1318 - ac

####P4.2b entrenando con tensorboard (opcional)

#####Activando TENSORBOARD 

---
(DASHBOARD para ver el proceso de entrenamiento)

In [19]:
# You can change the directory name
LOG_DIR = '/content/gdrive/MyDrive/Colab Notebooks//checkpoints_test/'

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-11-24 17:27:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  13.7MB/s    in 1.0s    

2021-11-24 17:27:41 (13.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [20]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=BATCH_SIZE,
                         write_images=True)

##P5. Generando texto nuevo usando la RNN

In [21]:
#creamos un modelo tomando como base el ultimo checkpoint
model = create_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))##tomamos el ultimo checkpoint
model.build(tf.TensorShape([1,None]))

In [22]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.2  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))

###P5.1 generando texto 

In [23]:
print(generate_text(model, start_string=u"diamantes de sangre"))

diamantes de sangrena de largos que hacia en la piscina todavia le que que que una joven repartia auriculares entre los invitados dema no lo se llevo todo el dia intentando localizar un trabajo que lo habia obligado a recorrer hombre apreto con mas fuerza el brazo de avila al tiempo que volvia a daritanico me llamo winston y supone un honor para mismo que debia de tratarse de una terrible pesadilla de la puerta del bar avila se llevo la tonica a continuacion habia seguiza a los matones vio en el espejo como el ofi


##P6.exportando modelo

---
Guardamos y Serializamos el Modelo (con esto ya podemos vender nuestro modelo de predicción de texto según lo aprendido por nuestra RNN).


In [24]:
from keras.models import model_from_json
import os
dir_export= '/content/gdrive/MyDrive/Colab Notebooks/'
#dir_export= os.path.join(dir_drive)
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'RNN_Origen_json.json'), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'RNN_Origen_pesos.hdf5'))
model.save(os.path.join(dir_export,'RNN_Origen_model.h5'))
print("modelo salvado en Drive de google")

modelo salvado en Drive de google
